****
# Import modules
****

In [1]:
import os
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf

****
# Load Data
****

In [2]:
file_path = "C:\\Users\\kruu\\store\\data_LSGG\\"

data_LSGG = pd.read_parquet(os.path.join(file_path + "landing_df_LSGG_with_meteo_0_of_22.parquet"))

for i in range(1, 22):
    temp = pd.read_parquet(os.path.join(file_path + f"landing_df_LSGG_with_meteo_{i}_of_22.parquet"))
    data_LSGG = pd.concat((data_LSGG, temp))
    

In [ ]:
data_LSGG[["avg_wind_dir", "avg_wind_speed", "avg_vis", "avg_temp", "avg_press"]].isna().any()

In [4]:
data_LSGG["nominal_distance_prop"] = data_LSGG["distance"] / data_LSGG["nominal_distance"] 

In [5]:
# STAR selection for the model
selected_stars = ["BELU3N", "KINE2N", "AKIT3R", "LUSA2N"]
lr_data = data_LSGG.query(f"star in {selected_stars}")

In [ ]:
# Redefinition of rush hours according to: https://www.slotcoordination.ch/public/upload/assets/875/gvas24sos.pdf?fp=1
# 7 - 10 / 11 -13 / 14 - 17 / 20 - 21

import pytz

def is_rush_hour(date):
    
    # Extract hour everything in UTC
    hour = date.hour
    minute = date.minute
    time_in_minutes = hour * 60 + minute
    
    if (7 * 60 <= time_in_minutes <= 10 * 60) or \
       (11 * 60 <= time_in_minutes <= 13 * 60) or \
       (14 * 60 <= time_in_minutes <= 17 * 60) or \
       (20 * 60 <= time_in_minutes <= 21 * 60):
        return True
    else:
        return False

lr_data['rush_hour'] = lr_data["start"].apply(is_rush_hour)


In [ ]:
# aircraft sizes

ac_body = {
    "Widebody": ["B763", "A333"],
    "Narrowbody": ["A320", "BCS3", "A20N", "A319", "BCS1", "A21N", "B738", "A321"],
    "Regional Jet": ["E190", "CRJ9", "E195", "DH8D"],
    "Business jets": ["PC12", "C56X", "F2TH", "PC24", "C68A", "E55P"],
}

def body_type(typecode, ac_body_dict):
    for key, value in ac_body_dict.items():
        if typecode in value:
            return key
    return None

lr_data[["body_type"]] =  lr_data.typecode.apply(lambda x: pd.Series(body_type(x, ac_body)))

****
# LR Model
****

In [ ]:
boxplot = lr_data.boxplot(["nominal_distance_prop"], by = ["star"],
                     figsize = (16, 9),
                     vert = False,
                     showmeans = False,
                     notch = False,
                     whis = (2.5,97.5))

boxplot.axvline(x=1, color='darkorange', linestyle=':', linewidth=2)

custom_labels = ['AKIT3R', 'BELU3N', 'KINE2N', 'LUSA2N']
boxplot.set_yticklabels(custom_labels, fontsize=22, rotation=45)

boxplot.get_figure().suptitle('')
boxplot.set_ylabel("")
boxplot.set_title('')

boxplot.set_axisbelow(True)
boxplot.grid(True, linestyle='--', alpha=0.7)
boxplot.spines['top'].set_visible(False)
boxplot.spines['right'].set_visible(False)
boxplot.spines['left'].set_visible(False)
boxplot.spines['bottom'].set_visible(False)
boxplot.tick_params(axis='x', which='both', length=0, labelsize=20)

- If one were to model the categorical star variable as a fixed effect that model would assume the group means (one from each star) are independent from each other and shere the residul variance.
- if the categorical star variable is modeled as a random effect (random intercept only) the model would assume that the star measured are a sample of a larger population of stars that has its own mean and variance 

Not completely sure about modelling with LMM. The random effect intercept (for instance), gives us a confidence interval for the intercepts of all stars. 

Example: we have here a variance for the intercept of 0.019 i.e. an std of 0.14. It means that at 95%, the intercept of each star is located in 0.865 +/- 3*0.14. 

Do we really need that ? We can just model the STAR as categorical variables and make the assumption (maybe a bit strong but anyway) that the observatiosn are independent among the different stars (the covariates affect the distance proportion equally for all stars)

In [ ]:
model_lm = smf.ols(
    "nominal_distance_prop ~ C(star) + C(weekday) + avg_vis + avg_wind_speed + C(season) + avg_vis + avg_press + C(body_type, Treatment(reference='Narrowbody')) + C(rush_hour) + avg_temp", 
    lr_data, 
)

res_lm = model_lm.fit()
res_lm.summary()

In [ ]:
print(res_lm.summary().as_latex())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

coefficients = res_lm.params
p_values = res_lm.pvalues

# features = coefficients.index  # Feature names
features = np.array([ #Same order as coefficients.index 
    "Intercept", 
    "STAR: BELU3N",
    "STAR: KINE2N",
    "STAR: LUSA2N",
    "WEEKDAY: Tuesday",
    "WEEKDAY: Wednesday",
    "WEEKDAY: Thursday",
    "WEEKDAY: Friday",
    "WEEKDAY: Saturday",
    "WEEKDAY: Sunday",
    "SEASON: Spring",
    "SEASON: Summer",
    "SEASON: Winter",
    "BODY TYPE: Business jet",
    "BODY TYPE: Regional jet",
    "BODY TYPE: Widebody",
    "RUSH HOUR: True",
    "Visibility",
    "Wind speed",
    "Pressure",
    "Temperature",
])
importance = coefficients.values  # Absolute coefficients to represent feature importance

indices = np.argsort(importance)

bar_color = 'steelblue'

def significance_stars(p_value):
    if p_value <= 0.01:
        return '***'
    elif p_value <= 0.05:
        return '**'
    elif p_value <= 0.10:
        return '*'
    else:
        return ''

plt.figure(figsize=(12, 8))
plt.barh(range(len(importance)), importance[indices], align='center', color=bar_color)

for i in range(len(importance)):
    pvalue = significance_stars(p_values[indices][i])
    if p_values[indices][i] < 0.05: 
        if importance[indices][i] > 0:
            plt.text(importance[indices][i] * 1.01, i, f'{np.round(importance[indices][i],4)}{pvalue}', va='center', fontsize=10)
        else:
            plt.text(0.001, i, f'{np.round(importance[indices][i],4)}{pvalue}', va='center', fontsize=10)
        

y_labels = plt.yticks(range(len(importance)), features[indices], fontsize=11)
for i in range(len(importance)):
    if p_values[indices][i] > 0.05:
        y_labels[1][i].set_color('firebrick')

ax = plt.gca()
ax.grid(True, axis='y', linestyle='--', alpha=0.7)
ax.set_axisbelow(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.tick_params(axis='y', which='both', length=0)

plt.xlabel('Coefficient Magnitude', fontsize=14, labelpad=10)
plt.title('Feature Importance for LSGG with Significance Levels', fontsize=16, pad=15)
plt.tight_layout()
plt.show()


****
# Linear mixed model
****

In [ ]:
# The effect of these predictors is assumed to be constant across all observations.
fixed_effect = "nominal_distance_prop ~ C(body_type, Treatment(reference='Narrowbody')) + C(weekday) + avg_vis + avg_wind_speed + avg_press + rush_hour" 

# Randome effect at STAR level
random_effect = "~ avg_vis + avg_wind_speed + C(weekday)"

model_mixed = smf.mixedlm(
    fixed_effect, 
    lr_data,
    groups = "star", 
    re_formula=random_effect
)

res_mixedlm = model_mixed.fit()
res_mixedlm.summary()

Random intercept model:
- Variance accross stars = 0.209 (std = sqrt(0.209) = 0.457). This is the variation of the intercept accross stars. On average, the proportion vary about 45 % accross the stars under investigation. I guess here it's a bit meaningless as we don't consider that the 4 stars are among a bigger pool of stars. What we want is the exact variations from one star to another, which is given by the regular linear regression. 
- There is no mixed effect for the avg_vis and the avg_wind speed, meaning that those factors are only fixed effect. It is nice to mention. Basically, it means that across the 4 stars under investigation, 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (16, 6))

sns.distplot(res_lm.resid, hist = False, kde_kws = {"shade" : True, "lw": 1}, fit = stats.norm, ax = ax1)
ax1.set_title("KDE Plot of Model Residuals (Blue) and Normal Distribution (Black)")
ax1.set_xlabel("Residuals")

sm.qqplot(res_lm.resid, dist = stats.norm, line = 's', ax = ax2)
ax2.set_title("Q-Q Plot")

# Null hypothesis rejected = normality of the residuals violated
labels = ["Statistic", "p-value"]
norm_res = stats.shapiro(res_lm.resid)
for key, val in dict(zip(labels, norm_res)).items():
    print(key, val)

In [ ]:
from statsmodels.stats.diagnostic import het_white

fig = plt.figure(figsize = (16, 9))

ax = sns.boxplot(x = res_lm.model.groups, y = res_lm.resid)

ax.set_title("Distribution of Residuals per STAR")
ax.set_ylabel("Residuals")
ax.set_xlabel("STAR")

# Null hypothesis rejected: homoskedasticity of the variance is violated
het_white_res = het_white(res_lm.resid, res_lm.model.exog)
labels = ["LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test p-value"]
for key, val in dict(zip(labels, het_white_res)).items():
    print(key, val)